Use this file as a template to copy and build notebooks

In [ ]:
#@title Mount drive and load libraries
import os
from google.colab import drive

drive.mount('/content/drive/')
path = '/content/drive/MyDrive/msc-project-mbalmf01'
os.chdir(path)

In [ ]:
def get_aa_embedding(sequence: str, model: str, tokenizer, max_length: int):
  sequence = ' '.join(sequence)
  encoded_input = tokenizer(sequence, padding='max_length', return_tensors='pt')
  model_output = model(**encoded_input)
  return model_output.last_hidden_state

device='cpu'
def aa_embeddings(seqs: list, model, tokenizer, max_length: int, device) -> list:
    seqs = [" ".join(list(sequence)) for sequence in seqs]
    ids = tokenizer.batch_encode_plus(seqs, add_special_tokens=True, padding="longest")
    input_ids = torch.tensor(ids['input_ids']).to(device)
    attention_mask = torch.tensor(ids['attention_mask']).to(device)
    with torch.no_grad():
        embedding_repr = model(input_ids=input_ids,attention_mask=attention_mask)
    ember = embedding_repr.last_hidden_state
    return ember

seqs = df['sequence_alignment_aa_heavy'].to_list()
max_length = max([len(seq) for seq in seqs])

seqs_chunked = chunks(seqs, len(seqs) // 500)

for num, seq in enumerate(seqs_chunked):
  embeddings = [get_aa_embedding(s, model, tokenizer, max_length) for s in seq]
  embeddings = torch.cat(embeddings, dim=0)
  filepath = f"/content/drive/MyDrive/msc-project-mbalmf01/embeddings/230814_ablang_H_aa_embeddings_{num}.pt"
  torch.save(embeddings, filepath)
  del embeddings
  gc.collect()

l = []
for j in os.listdir('/content/drive/MyDrive/msc-project-mbalmf01/embeddings/'):
  t = torch.load(f'/content/drive/MyDrive/msc-project-mbalmf01/embeddings/{j}')
  t = t.detach().numpy()
  l.append(t)

ten = np.concatenate(l)

!cp /content/array2.npy /content/drive/MyDrive/msc-project-mbalmf01/embeddings/array.npy